List avaialable atac-seq files

In [1]:
!ls -lt /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/*ATAC*.fastq*

-rw-rw---- 1 marinovg oak_akundaje          11 Jun  3 18:39 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC.barcodes_annotated.fastq.lines
-rw-rw---- 1 marinovg oak_akundaje 15208559316 Jun  3 00:03 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC.barcodes_annotated.end2.fastq.gz
-rw-rw---- 1 marinovg oak_akundaje 15809189887 Jun  3 00:03 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC.barcodes_annotated.end1.fastq.gz
-rw-rw---- 1 marinovg oak_akundaje 10494576406 Jun  2 15:03 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC_L002_R2.fastq.gz
-rw-rw---- 1 marinovg oak_akundaje  9104962588 Jun  2 15:00 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC_L001_R2.fastq.gz
-rw-rw---- 1 marinovg oak_akundaje  6434043860 Ju

Create directory for the data (once)

In [6]:
# !mkdir atac-seq

Run the pipeline for one lane (to save time)

In [2]:
!ls -lt /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/*ATAC_L001*.fastq*

-rw-rw---- 1 marinovg oak_akundaje 9104962588 Jun  2 15:00 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC_L001_R2.fastq.gz
-rw-rw---- 1 marinovg oak_akundaje 5589440271 Jun  2 14:52 /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/BMMC_single_donor_ATAC_L001_R1.fastq.gz


Copy the atac files locally (once)

In [7]:
# !cp /oak/stanford/groups/akundaje/marinovg/ENCODE4/single-cell/2023-06-02-scJamboree/*ATAC_L001*.fastq* atac-seq/

Run the aligner
- install - check that the version is > 0.2.4 (there was issue with creating a valid BAM file in earlier versions)
- installation command is: conda install -c bioconda -c conda-forge chromap
- check verion (above 0.2.4.)

In [9]:
!chromap -v

0.2.5-r473


Create a index file (once)
- use reference file that you have locally on the machien that you are working on

Create index folder for index file (once)

In [10]:
# !mkdir chromap_index

In [13]:
# run once
# !chromap -i -r /users/eila/git_kundaje/RNA/single-cell/expression/kb_seqspec/reference/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -o chromap_index/index



Build index for the reference.
Kmer length: 17, window size: 7
Reference file: /users/eila/git_kundaje/RNA/single-cell/expression/kb_seqspec/reference/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
Output file: chromap_index/index
Loaded all sequences successfully in 20.38s, number of sequences: 194, number of bases: 3099750718.
Collected 739180678 minimizers.
Sorting minimizers.
Sorted all minimizers.
Kmer size: 17, window size: 7.
Lookup table size: 393121381, # buckets: 536870912, occurrence table size: 447341405, # singletons: 291839273.
Built index successfully in 298.77s.
[M::Statistics] kmer size: 17; skip: 7; #seq: 194
[M::Statistics::3.826] distinct minimizers: 393121381 (74.24% are singletons); average occurrences: 1.880; average spacing: 4.193
Saved in 111.19s.


In [ ]:
# run with --preset atac --drop-repetitive-reads 4 -q 0

#-t 4 (number of threads)
# --summary FILE       Summarize the mapping statistics at bulk or barcode level. VERY HELPFUL TO SEE WHATS GOING ON
# --preset STR              Preset parameters for mapping reads (always applied before other options) []
#                                 atac: mapping ATAC-seq/scATAC-seq reads
#                                 chip: mapping ChIP-seq reads
#                                 hic: mapping Hi-C reads

In [15]:
!ls atac-seq

BMMC_single_donor_ATAC_L001_R1.fastq.gz
BMMC_single_donor_ATAC_L001_R2.fastq.gz


In [17]:
# !mkdir SAM_out

In [2]:
# check how many cores are available. choose threads accordingly
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              48
On-line CPU(s) list: 0-47
Thread(s) per core:  2
Core(s) per socket:  12
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz
Stepping:            2
CPU MHz:             2593.780
CPU max MHz:         2600.0000
CPU min MHz:         1200.0000
BogoMIPS:            5187.69
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            30720K
NUMA node0 CPU(s):   0-11,24-35
NUMA node1 CPU(s):   12-23,36-47
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dte

In [3]:
!time chromap -x chromap_index/index --preset atac --drop-repetitive-reads 4 -q 0 -t 16 \
-r /users/eila/git_kundaje/RNA/single-cell/expression/kb_seqspec/reference/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz \
-1 atac-seq/BMMC_single_donor_ATAC_L001_R1.fastq.gz, atac-seq/atac-seq/BMMC_single_donor_ATAC_L001_R2.fastq.gz --SAM -o SAM_out/alignment.sam --summary SAM_out/SAM_summary.txt  


Sorted, deduped and outputed mappings in 399.36s.
# uni-mappings: 83010931, # multi-mappings: 5509346, total: 88520277.
Number of output mappings (passed filters): 88520277
Total time: 1587.81s.

real	26m37.466s
user	277m23.644s
sys	5m6.591s


Preset parameters for ATAC-seq/scATAC-seq are used.
Start to map reads.
Parameters: error threshold: 8, min-num-seeds: 2, max-seed-frequency: 500,1000, max-num-best-mappings: 1, max-insert-size: 2000, MAPQ-threshold: 0, min-read-length: 30, bc-error-threshold: 1, bc-probability-threshold: 0.90
Number of threads: 16
Analyze bulk data.
Will try to remove adapters on 3'.
Will remove PCR duplicates after mapping.
Will remove PCR duplicates at cell level.
Won't allocate multi-mappings after mapping.
Only output unique mappings after mapping.
Only output mappings of which barcodes are in whitelist.
Perform Tn5 shift.
Output mappings in SAM format.
Reference file: /users/eila/git_kundaje/RNA/single-cell/expression/kb_seqspec/reference/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
Index file: chromap_index/index
1th read 1 file: atac-seq/BMMC_single_donor_ATAC_L001_R1.fastq.gz
Output file: SAM_out/alignment.sam
Loaded all sequences successfully in 20.09s, number of sequences: 194, number of bases: 3099750718.
Kmer size: 17, window size: 7.
Lookup table size: 393121381, occurrence table size: 447341405.
Loaded index successfully in 106.70s.
Loaded 500000 reads in 0.68s.
Loaded 500000 reads in 2.63s.

In [ ]:
# question: why the output includes: Analyze bulk data.

Convert the sam file to a bam file

install samtools
- conda install -c bioconda samtools


In [ ]:
# run once
# !samtools view -bS SAM_out/alignment.sam > SAM_out/alignment.bam

Check that the BAM file is valid

In [2]:
!ls SAM_out/

alignment.bam  alignment.sam  SAM_summary.txt


In [5]:
!samtools flagstat SAM_out/alignment.bam

88520277 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
88520277 + 0 mapped (100.00% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
